Using a GRU network for wildfire predictions

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

from sklearn.preprocessing import StandardScaler


2024-05-31 14:47:06.192314: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 14:47:07.717657: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')


In [3]:
!pip install netcdf4 tensorflow

/opt/conda/envs/python3/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [4]:
#ERA5 = xr.open_dataset('/content/drive/MyDrive/Data/merge_fixed.nc', engine = 'netcdf4')

ERA5 = xr.open_dataset('merge_fixed.nc', engine = 'netcdf4')


In [5]:
# write down all of the variable names
# Only use the second ones for the model
var_names = ['cape', 'cp', 'cvh', 'cvl', 'e', 'fraction_of_burnable_area', 'msl', 'ssr', 'swvl1', 't2m', 'tcc', 'tp', 'u10', 'v10']
var_names = ['cape', 'cvh', 'cvl', 'e', 'msl', 'ssr', 't2m', 'tcc']

target_name = 'burned_area'
long_names = ['Convective available potential energy', 'Convective precipitation', 'High vegetation cover', 'Low vegetation cover', 'Evaporation', 'fraction of burnable area', 'latitude', 'longitude', 'Mean sea level pressure', 'Surface net short-wave (solar) radiation', 'Sea surface temperature', 'Volumetric soil water layer 1', '2 metre temperature', 'Total cloud cover', 'time', 'Total precipitation', '10 metre U wind component', '10 metre V wind component']

In [6]:
# Stack the latitude and longitude dimensions into a single grid-point dimension
ERA5_stacked = ERA5.stack(grid_point=("latitude", "longitude"))

# Now 'data_stacked' has dimensions (time, grid_point)
print(ERA5_stacked)

# reset the grid-point index to create a more interpretable structure
ERA5_stacked = ERA5_stacked.reset_index("grid_point")

<xarray.Dataset> Size: 4GB
Dimensions:                    (time: 228, grid_point: 150801)
Coordinates:
  * time                       (time) datetime64[ns] 2kB 2001-01-01 ... 2019-...
  * grid_point                 (grid_point) object 1MB MultiIndex
  * latitude                   (grid_point) float32 603kB 90.0 90.0 ... 15.0
  * longitude                  (grid_point) float32 603kB -170.0 ... -45.0
Data variables: (12/16)
    burned_area                (time, grid_point) float32 138MB nan nan ... nan
    cape                       (time, grid_point) float64 275MB 0.7234 ... 174.6
    cp                         (time, grid_point) float64 275MB 1.129e-06 ......
    cvh                        (time, grid_point) float64 275MB 0.0 0.0 ... 0.0
    cvl                        (time, grid_point) float64 275MB 0.0 0.0 ... 0.0
    e                          (time, grid_point) float64 275MB -8.911e-06 .....
    ...                         ...
    swvl1                      (time, grid_point) float

In [7]:
# Do standard scaling on the Dataset

# Initialize the StandardScaler
scaler = StandardScaler()

# Dictionary to store the scaled data
ERA5_scaled = ERA5_stacked

# Loop through each variable in the Dataset
for var in var_names:

    print(var)

    # Apply StandardScaler
    data_scaled = scaler.fit_transform(ERA5_stacked[var].values)

    # Reshape back to the original structure
    data_scaled = data_scaled.reshape(ERA5_stacked[var].shape)

    ERA5_scaled[var].values = data_scaled


cape
cvh
cvl
e
msl
ssr
t2m
tcc


In [8]:
# Create a mask for grid points where burned_area is greater than 0 at any time
mask = ERA5_stacked['burned_area'].max(dim='time') > 0

# Use the mask to select grid points
filtered_ERA5 = ERA5_stacked.sel(grid_point=mask)

filtered_ERA5

<xarray.Dataset> Size: 817MB
Dimensions:                    (time: 228, grid_point: 29866)
Coordinates:
  * time                       (time) datetime64[ns] 2kB 2001-01-01 ... 2019-...
    latitude                   (grid_point) float32 119kB 69.75 69.75 ... 15.25
    longitude                  (grid_point) float32 119kB -128.2 ... -61.25
Dimensions without coordinates: grid_point
Data variables: (12/16)
    burned_area                (time, grid_point) float32 27MB 0.0 0.0 ... 0.0
    cape                       (time, grid_point) float64 54MB -0.4677 ... -0...
    cp                         (time, grid_point) float64 54MB 5.643e-06 ... ...
    cvh                        (time, grid_point) float64 54MB 1.665e-16 ... 0.0
    cvl                        (time, grid_point) float64 54MB 2.22e-16 ... 0.0
    e                          (time, grid_point) float64 54MB 0.8361 ... -0....
    ...                         ...
    swvl1                      (time, grid_point) float64 54MB 0.2586 ... 9.3...
    t2m                        (time, grid_point) float64 54MB -1.086 ... 0.1682
    tcc                        (time, grid_point) float64 54MB 0.06741 ... -0...
    tp                         (time, grid_point) float64 54MB 0.0005318 ... ...
    u10                        (time, grid_point) float64 54MB -1.504 ... -7.68
    v10                        (time, grid_point) float64 54MB 0.539 ... -0.3231
Attributes: (12/38)
    CDI:                        Climate Data Interface version 2.0.5 (https:/...
    Conventions:                CF-1.6
    title:                      Fire_cci Gridded MODIS Burned Area product
    history:                    Tue May 28 11:43:39 2024: ncks -C -O -x -v ti...
    references:                 See www.esa-fire-cci.org
    tracking_id:                9b5d5715-a09f-4db6-89e9-c63833929d3c
    ...                         ...
    geospatial_lat_units:       degrees_north
    geospatial_lon_resolution:  0.25
    geospatial_lat_resolution:  0.25
    id:                         20010101-ESACCI-L4_FIRE-BA-MODIS-fv5.1.1cds.nc
    CDO:                        Climate Data Operators version 2.0.5 (https:/...
    NCO:                        netCDF Operators version 5.0.1 (Homepage = ht...

In [9]:
sequence_number = 4
feature_number = len(var_names)

X = np.empty((228 - sequence_number, sequence_number + 1, 29866, feature_number))



In [10]:
for k in range(len(var_names)):
  variable = var_names[k]
  print(variable)
  for t in range(sequence_number + 1):
    X[:, t, :, k] = np.roll(filtered_ERA5[str(variable)], t, axis = 0)[sequence_number:,:]



cape
cvh
cvl
e
msl
ssr
t2m
tcc


In [11]:
y = filtered_ERA5.burned_area[sequence_number:,:]

In [12]:
time_max = 12*6

In [13]:
X = X[0:time_max, :, :, :]
y = y[0:time_max, :]

In [14]:
# Check for NaN values in the data
assert not np.any(np.isnan(X)) or np.any(np.isinf(X)), "Input data contains unusable values"
assert not np.any(np.isnan(y)) or np.any(np.isinf(y)), "Target data contains unusable values"

In [17]:
X.shape

(72, 5, 29866, 8)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Example data dimensions
num_samples = time_max
num_timesteps = sequence_number + 1
num_gridcells = 29866
num_features = 8

# Reshape the data to process each grid cell independently
X_reshaped = X.reshape(num_samples, num_timesteps, num_gridcells * num_features)

# Define the simplest GRU model
model = Sequential()
model.add(InputLayer(input_shape=(num_timesteps, num_gridcells * num_features)))
model.add(GRU(units=16, return_sequences=False))  # Minimal units to reduce memory usage
model.add(Dense(units=num_gridcells, activation='linear'))

# Compile the model with a lower learning rate and gradient clipping
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Model summary
model.summary()

# Train-test split (80-20 split)
train_size = int(0.8 * num_samples)
X_train, X_test = X_reshaped[:train_size], X_reshaped[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with a smaller batch size to reduce memory usage
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
test_loss = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)

# Make predictions
y_pred = model.predict(X_test)

# Compute additional metrics (e.g., MAE, RMSE)
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('MAE:', mae)
print('RMSE:', rmse)

/opt/conda/envs/python3/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 16)             │    11,469,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 29866)          │       507,722 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,977,130 (45.69 MB)

 Trainable params: 11,977,130 (45.69 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - loss: 6400301858816.0000 - val_loss: 5627756675072.0000
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - loss: 6625349337088.0000 - val_loss: 5627756675072.0000
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - loss: 6100175814656.0000 - val_loss: 5627756675072.0000
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 6532109434880.0000

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, TimeDistributed, Flatten, GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Example data dimensions
num_samples = time_max - sequence_number
num_timesteps = sequence_number + 1
num_gridcells = 29866
num_features = feature_number

# Define the simplest model with Conv2D and GRU
model = Sequential()

# Convolutional layer to capture spatial features
model.add(TimeDistributed(Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu'), 
                         input_shape=(num_timesteps, num_gridcells, num_gridcells, num_features)))
model.add(TimeDistributed(Flatten()))

# GRU layers to capture temporal dependencies
model.add(GRU(units=16, return_sequences=False))  # Minimal units to reduce memory usage

# Dense layer to output the prediction for each grid cell
model.add(Dense(units=num_gridcells * num_gridcells, activation='linear'))
model.add(tf.keras.layers.Reshape((num_gridcells, num_gridcells)))

# Compile the model with a lower learning rate and gradient clipping
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Model summary
model.summary()

# Train-test split (80-20 split)
train_size = int(0.8 * num_samples)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with a smaller batch size to reduce memory usage
history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
test_loss = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)

# Make predictions
y_pred = model.predict(X_test)




In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, TimeDistributed, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


# Define a simpler model
model = Sequential()
model.add(TimeDistributed(Dense(8), input_shape=(num_timesteps, num_gridcells, num_features)))
model.add(TimeDistributed(Flatten()))
model.add(GRU(units=16, return_sequences=True, recurrent_dropout=0.2))
model.add(GRU(units=8, recurrent_dropout=0.2))
model.add(Dense(units=num_gridcells, activation='linear'))

# Compile the model with a lower learning rate and gradient clipping
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Model summary
model.summary()

# Train-test split (80-20 split)
train_size = int(0.8 * num_samples)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with a smaller batch size to reduce memory usage
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
test_loss = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)

# Make predictions
y_pred = model.predict(X_test)

# Compute additional metrics (e.g., MAE, RMSE)
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('MAE:', mae)
print('RMSE:', rmse)